In [1]:
from __future__ import division, print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from tempfile import mkdtemp, mkstemp
from os.path import join
from subprocess import call, check_output, Popen, PIPE

Original Code

In [2]:
def feature_frequency_orig(melFTmix, TR):
    """ This function extracts the high-frequency content feature scores. It determines the frequency, as fraction of the Nyquist frequency, at which the higher and lower frequencies explain half of the total power between 0.01Hz and Nyquist. 

    Parameters
    ---------------------------------------------------------------------------------
    melFTmix: Full path of the melodic_FTmix text file
    TR: TR (in seconds) of the fMRI data (float)

    Returns
    ---------------------------------------------------------------------------------
    HFC: Array of the HFC ('High-frequency content') feature scores for the components of the melodic_FTmix file"""

    # Determine sample frequency
    Fs = 1/TR

    # Determine Nyquist-frequency
    Ny = Fs/2

    # Load melodic_FTmix file
    FT=np.loadtxt(melFTmix)

    # Determine which frequencies are associated with every row in the melodic_FTmix file  (assuming the rows range from 0Hz to Nyquist)
    f = Ny*(np.array(range(1,FT.shape[0]+1)))/(FT.shape[0])

    # Only include frequencies higher than 0.01Hz
    fincl = np.squeeze(np.array(np.where( f > 0.01 )))
    FT=FT[fincl,:]
    f=f[fincl]

    # Set frequency range to [0-1]
    f_norm = (f-0.01)/(Ny-0.01)

    # For every IC; get the cumulative sum as a fraction of the total sum
    fcumsum_fract = np.cumsum(FT,axis=0)/ np.sum(FT,axis=0)

    # Determine the index of the frequency with the fractional cumulative sum closest to 0.5
    idx_cutoff=np.argmin(np.abs(fcumsum_fract-0.5),axis=0)

    # Now get the fractions associated with those indices index, these are the final feature scores
    HFC = f_norm[idx_cutoff]

    # Return feature score
    return HFC

In [3]:
TR = 2.0
ftmix = os.path.abspath('../test/refout/melodic.ica/melodic_FTmix')
hfc_orig = feature_frequency_orig(melFTmix=ftmix, TR=TR)

Refactored version

In [4]:
def feature_frequency(ftmix, t_r):
    """High-frequency content feature scores.

    It determines the frequency, as fraction of the Nyquist frequency, at which the higher and lower
    frequencies explain half of the total power between 0.01Hz and Nyquist.

    Parameters
    ----------
    ftmix: rank 2 numpy array
        melodic ft mix array
    t_r: float
        Repetition time (in seconds) of the fMRI data

    Returns
    -------
    rank 1 numpy.array
        HFC ('High-frequency content') feature scores for the components of the melodic_FTmix file
    """
    assert ftmix.ndim == 2
    assert 0.5 < t_r < 10

    sample_frequency = 1 / t_r
    nyquist = sample_frequency / 2

    # Determine which frequencies are associated with every row in the melodic_FTmix file
    # (assuming the rows range from 0Hz to Nyquist)
    # TODO: RHD: How many rows? Off by one? is the first row 0Hz or nyquist/n and the last (n-1)/n * nyquist or nyquist?
    frequencies = nyquist * (np.arange(ftmix.shape[0]) + 1) / ftmix.shape[0]

    # Include only frequencies above 0.01 Hz
    ftmix = ftmix[frequencies > 0.01, :]
    frequencies = frequencies[frequencies > 0.01]

    # Set frequency range to [0, 1]
    normalised_frequencies = (frequencies - 0.01) / (nyquist - 0.01)

    # For every IC; get the cumulative sum as a fraction of the total sum
    fcumsum_fraction = np.cumsum(ftmix, axis=0) / np.sum(ftmix, axis=0)

    # Determine the index of the frequency with the fractional cumulative sum closest to 0.5
    index_cutoff = np.argmin((fcumsum_fraction - 0.5)**2, axis=0)

    # Now get the fractions associated with those indices index, these are the final feature scores
    hfc = normalised_frequencies[index_cutoff]

    # Return 'High-frequency content' feature score
    return hfc

In [5]:
hfc = feature_frequency(ftmix=np.loadtxt(ftmix), t_r=TR)
assert np.allclose(hfc, hfc_orig)

In [6]:
%timeit feature_frequency_orig(melFTmix=ftmix, TR=TR)
%timeit feature_frequency(ftmix=np.loadtxt(ftmix), t_r=TR)

100 loops, best of 3: 6.22 ms per loop
100 loops, best of 3: 5.71 ms per loop


No real difference in timing, but that's to be expected - the changes here are only cosmetic. The contribution to the profiled time is pretty small here as well.